Импорты и настройки

In [58]:
import json
import subprocess
import re
from pathlib import Path
import pandas as pd
from IPython.display import display
from tqdm.notebook import tqdm  # прогрессбар для Jupyter

# Путь до папки data/, где лежит test/ann
DATASET_ROOT = Path("../../data")

# Подпапки с аннотациями
ANN_DIRS = ["test/ann"]

# Настройка модели и промпта
MODEL_NAME = "mistral"
PROMPT_TEMPLATE = (
    "На входе — текст с автомобильного номера: '{plate_text}'. "
    "Верни номер строго в формате ГОСТ: [буква][3 цифры][2 буквы][регион, 2–3 цифры]. "
    "Никаких пробелов, дефисов и пояснений — только номер. "
    "Если это невозможно, верни пустую строку."
)


In [ ]:
# list((DATASET_ROOT / "test/ann").glob("*.json"))
for ann_subdir in ANN_DIRS:
    dir_path = DATASET_ROOT / ann_subdir
    if dir_path.exists():
        for old in dir_path.glob("*_fixed.json"):
            old.unlink()
            print(f"🗑 Удалён: {old.name}")


Работа с LLM и фильтрациия по ГОСТ

In [65]:
def query_ollama(prompt, model=MODEL_NAME):
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    return result.stdout.decode("utf-8").strip()

def clean_response(response: str) -> str:
    if "'" in response:
        parts = response.split("'")
        if len(parts) >= 2:
            return parts[1]
    return response.strip()

def enforce_gost_format(plate: str) -> str:
    plate = re.sub(r"[^\w]", "", plate).upper()
    if re.match(r"^[АВЕКМНОРСТУХA-Z]{1}\d{3}[АВЕКМНОРСТУХA-Z]{2}\d{2,3}$", plate):
        return plate
    return ""


Обработка одного файла и сбор результатов

In [66]:
def process_json_file(json_path: Path):
    with json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    plate_text = data.get("description", "").strip()
    mod = data.get("moderation", {})
    current_pred = mod.get("predicted", "").strip()

    prompt = PROMPT_TEMPLATE.format(plate_text=plate_text)
    llm_response = query_ollama(prompt)
    fixed_pred_raw = clean_response(llm_response)
    fixed_pred = enforce_gost_format(fixed_pred_raw)

    # Флаги
    was_overwritten = fixed_pred != ""
    was_changed = was_overwritten and fixed_pred != current_pred

    # Обновление
    if was_overwritten:
        data["moderation"]["predicted"] = fixed_pred
        if not data["moderation"].get("moderatedBy"):
            data["moderation"]["moderatedBy"] = "llm-auto"

        output_path = json_path.with_name(json_path.stem + "_fixed.json")
        with output_path.open("w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

    return {
        "file": json_path.name,
        "plate_text": plate_text,
        "old_moderation_predicted": current_pred,
        "llm_predicted": fixed_pred,
        "was_changed": was_changed,
        "was_overwritten": was_overwritten
    }


Проход по всем файлам и итоговая таблица

In [67]:
all_changes = []

# Считаем общее количество файлов
total_files = sum(
    len(list((DATASET_ROOT / subdir).glob("*.json")))
    for subdir in ANN_DIRS
    if (DATASET_ROOT / subdir).exists()
)

with tqdm(total=total_files, desc="Обработка аннотаций") as pbar:
    for ann_subdir in ANN_DIRS:
        dir_path = DATASET_ROOT / ann_subdir
        if not dir_path.exists():
            print(f"⚠️ Папка {dir_path} не найдена — пропускаем")
            continue

        for json_file in dir_path.glob("*.json"):
            changes = process_json_file(json_file)
            all_changes.append(changes)
            pbar.update(1)


Обработка аннотаций:   0%|          | 0/164 [00:00<?, ?it/s]

In [68]:
if all_changes:
    df = pd.DataFrame(all_changes)

    # Фильтры
    df_changed = df[df["was_changed"] == True].reset_index(drop=True)
    df_overwritten = df[df["was_overwritten"] == True].reset_index(drop=True)

    print(f"\n🔍 Изменено по сравнению с оригиналом: {len(df_changed)}")
    print(f"📝 Всего переписано (включая совпадения): {len(df_overwritten)}")

    # Показываем полную таблицу с LLM-результатами
    display(df_overwritten)
else:
    print("📭 Нет изменений или не найдено JSON-файлов.")



🔍 Изменено по сравнению с оригиналом: 2
📝 Всего переписано (включая совпадения): 37


,file,plate_text,old_moderation_predicted,llm_predicted,was_changed,was_overwritten
0,O002KC98.json,O002KC98,O002KC98,O002KC98,False,True
1,M204OH98_0.json,M204OH98,M204OH98,M204OH98,False,True
2,K998TB13.json,K998TB13,K998TB13,K998TB13,False,True
3,M851AC197_0.json,M851AC197,M851AC197,M851AC197,False,True
4,O001TK178.json,O001TK178,O001TK178,O001TK178,False,True
5,M605HY31_0.json,M605HY31,M605HY31,M605HY31,False,True
6,K700OX46.json,K700OX46,K700OX46,K700OX46,False,True
7,K960YX159.json,K960YX159,K960YX159,K960YX159,False,True
8,K956BP178.json,K956BP178,K956BP178,K956BP178,False,True
9,M168XK116_0.json,M168XK116,M168XK116,M168XK116,False,True
